<a href="https://colab.research.google.com/github/matiasmazzucchi/Bicycle-scraper/blob/main/Bicycle_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CODIGO PARA EXTRAER KROMOS, BERTOLLINA Y FACHIN BIKES.
---

In [ ]:
#llamar a las siguientes bibliotecas
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import csv
import datetime
import gspread


#URL DE CADA PAGINA (terminados en 'K'es de la pagina Kromos y terminados en 'B' es de la pagina de bertollina, terminado con 'F' es de la pagina de Fachin)
URL_BASE_K = 'https://www.kromosbicicletas.com.ar/productos/mar/1/venzo'
URL_BASE_B = 'https://www.bertolinabikes.com/search/?q=venzo&mpage=2'
URL_BASEF = 'https://www.facchinbicycles.com.ar/listado/deportes-fitness/ciclismo/bicicletas/venzo/'

#Obtener el HTML de la página
pedido_obtenidoK = requests.get(URL_BASE_K)
html_obtenidoK = pedido_obtenidoK.text
pedido_obtenidoB = requests.get(URL_BASE_B)
html_obtenidoB = pedido_obtenidoB.text
pedido_obtenidoF = requests.get(URL_BASEF)
html_obtenidoF = pedido_obtenidoF.text

#Parsear el HTML
soupK = BeautifulSoup(html_obtenidoK, 'html.parser')
soupB = BeautifulSoup(html_obtenidoB, 'html.parser')
soupF = BeautifulSoup(html_obtenidoF, 'html.parser')


In [ ]:
# Encontrar todos los títulos y precios
#KROMOS
bicicletasK = soupK.find_all('h2', class_='tt-title')
precioK = soupK.find_all('div', class_="tt-price")
#BERTOLLINA
bicicletasB = soupB.find_all('a', class_='js-item-name')
preciosB = soupB.find_all('span', class_='js-price-display item-price h6')
precios_viejosB = soupB.find_all('span', class_="js-compare-price-display item-price-compare p-none-left-xs p-right-quarter-xs")
#FACHIN
bicicletaF = soupF.find_all('h2', class_='ui-search-item__title shops__item-title')
preciosF = soupF.find_all('div', class_='ui-search-price ui-search-price--size-medium shops__price')


In [ ]:
#Creating the lists for the next dataframe
#KROMOS
bicicletasKL = []
precioKL = []
#BERTOLLINA
bicicletaBL = []
preciosBL = []
precios_viejosBL = []
#FACHIN
bicicletaLF = []
preciosLF = []

In [ ]:
#Ciclos for para limpiar los datos y obtener los nombres y bicicletas
#KROMOS
for bicicleta in bicicletasK:
 bicicletasKL.append(bicicleta.get_text(strip=True))

for precios in precioK:
  precioKL.append(precios.get_text(strip=True))

#BERTOLLINA
for bicicletas in bicicletasB:
  bicicletaBL.append(bicicletas.get_text(strip=True))

for precios in preciosB:
  preciosBL.append(precios.get_text(strip=True))


for precio_viejo in precios_viejosB:
  precios_viejosBL.append(precio_viejo.get_text(strip=True))

#FACHIN
for bicicletas in bicicletaF:
  bicicletaLF.append(bicicletas.get_text(strip=True))

for precio in preciosF:
  preciosLF.append(precio.get_text(strip=True))


In [ ]:
#Armado del dataframe
#KROMOS DATAFRAME
ListaFinalK = list(zip(bicicletasKL, precioKL))
dfK = pd.DataFrame(ListaFinalK, columns=['Bicicleta','Precio'])

#BERTOLLINA DATAFRAME
ListaFinalB = list(zip(bicicletaBL, preciosBL, precios_viejosBL))
dfB = pd.DataFrame(ListaFinalB, columns=['Bicicleta','Precio', 'Precios Viejos'])


# prompt: eliminar los puntos de la columna de 'Precios' y 'Precios Viejos' de la variable 'dfB'

dfB['Precios Viejos'] = dfB['Precios Viejos'].str.replace('.', '')
dfB['Precio'] = dfB['Precio'].str.replace('.', '')

# prompt: eliminar los puntos de la columna 'Precio' de la variable 'dfK' y tambien eliminar los ultimos dos digitos de la columna 'Precio' del la variable 'dfK'

# Eliminar los puntos de la columna 'Precio' de la variable 'dfK'
dfK['Precio'] = dfK['Precio'].str.replace('.', '')

# Eliminar los últimos dos dígitos de la columna 'Precio' del la variable 'dfK'
dfK['Precio'] = dfK['Precio'].str[:-2]

# prompt: Añadir una columna que se llame fecha de extaraxion  en el dfK y que registre la fecha en la que se realizo la extraccion del dato

# Get the current date and time
now = datetime.datetime.now()

# Format the date and time
formatted_date = now.strftime("%Y-%m-%d")

# Add a new column to the DataFrame
dfK["Fecha de extraccion"] = formatted_date
dfB["Fecha de extraccion"] = formatted_date

#FACHIN DATAFRAME
ListaFinalF = list(zip(bicicletaLF, preciosLF))
dfF = pd.DataFrame(ListaFinalF, columns=['Bicicleta','Precio'])

# prompt: si algun registro de la olumna 'Precio' contiene 'OFF' eliminar los primeros 8 caracteres en la variable 'dfF'

dfF['Precio'] = dfF['Precio'].apply(lambda x: x[8:] if 'OFF' in x else x)

# prompt: eliminar los puntos de la columna 'Precio' de la variable dfF  y tambien eliminar los caracteres igual a '%' y 'OFF'

dfF['Precio'] = dfF['Precio'].str.replace('.', '')
dfF['Precio'] = dfF['Precio'].str.replace('%', '')
dfF['Precio'] = dfF['Precio'].str.replace('OFF', '')

# prompt: añadir en el 'dfF' una columna que se llame 'fecha de extraccion' que contenga la fecha en la que se realizo la extraccion de los datos

# Get the current date and time
now = datetime.datetime.now()

# Format the date and time
formatted_date = now.strftime("%Y-%m-%d")

# Add a new column to the DataFrame
dfF["Fecha de extraccion"] = formatted_date







CREATING A DATAFRAME TO EXPORT DATA TO GOOGLE SHEETS FROM GOOGLE COLAB


In [ ]:
#Autenticacion y permisos de uso de la API de google SHEETS
from google.colab import auth
auth.authenticate_user()

from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

#CREACION de la hoja de caluclo de google sheets
sh = gc.create('Bicicletas Web scraping Gspread KROMOS, BERTOLLINA AND FACHIN')

#Crado de las hojas dentro de  'Bicicletas Web scraping Gspread KROMOS AND BERTOLLINA'

worksheet1 = gc.open('Bicicletas Web scraping Gspread KROMOS, BERTOLLINA AND FACHIN').add_worksheet(title="Kromos", rows=100, cols=20)
worksheet2 = gc.open('Bicicletas Web scraping Gspread KROMOS, BERTOLLINA AND FACHIN').add_worksheet(title="Bertollina", rows=100, cols=20)
worksheet3 = gc.open('Bicicletas Web scraping Gspread KROMOS, BERTOLLINA AND FACHIN').add_worksheet(title="Fachin", rows=100, cols=20)

#Agregado del Open de las distintas hojas
worksheetK = gc.open('Bicicletas Web scraping Gspread KROMOS, BERTOLLINA AND FACHIN').worksheet('Kromos')
worksheetB = gc.open('Bicicletas Web scraping Gspread KROMOS, BERTOLLINA AND FACHIN').worksheet('Bertollina')
worksheetF = gc.open('Bicicletas Web scraping Gspread KROMOS, BERTOLLINA AND FACHIN').worksheet('Fachin')

#Apertura de todas las hojas y almacenamiento de los datos en las hojas correspondientes
worksheetK.update([dfK.columns.values.tolist()] + dfK.values.tolist())
worksheetB.update([dfB.columns.values.tolist()] + dfB.values.tolist())
worksheetF.update([dfF.columns.values.tolist()] + dfF.values.tolist())



---



In [ ]:
#ESTE ES EL CODIGO COMPLETO PARA DESCARGAR DIRECTAMENTE EL CSV Y HACERLO CON LOS METODOS VIEJOS
#bicicletasL.insert(0, "Bicicletas")
#precioL.insert(0, "Precio")
#datos = dict(zip(bicicletasL, precioL))
#with open ("datos.csv", "w") as f:
 # w = csv.writer(f)
 # w.writerows(datos.items())



---

